In [24]:
# import sys
# sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks/OCR/src')

# import function from py file
# from utils.py import show_img

In [ ]:
# run python files
# !python /content/drive/MyDrive/Colab\ Notebooks/OCR/src/model_definition.py

In [30]:
import os
import random
from sklearn.model_selection import train_test_split

Get list of images

In [37]:
IMG_LIST_DIR = '/content/drive/MyDrive/Colab Notebooks/OCR/data/PageImg/'
SEG_LIST_DIR = '/content/drive/MyDrive/Colab Notebooks/OCR/data/PageSeg/'

image_list = os.listdir(IMG_LIST_DIR)
image_list = [filename.split(".")[0] for filename in image_list]

seg_list = os.listdir(SEG_LIST_DIR)
seg_list = [filename.split(".")[0] for filename in seg_list]

Split data into train and test

In [38]:
X_train, X_test, y_train, y_test = train_test_split(image_list, seg_list, test_size = 0.3, shuffle = True)

Import model

In [ ]:
# model=unet()
# model.summary()